In [7]:
import os
import sys
import pickle
from typing import Optional, Union

import numpy
import pandas

__CW = os.path.dirname(os.path.abspath("")) + "/notebooks/"
__SRC_FOLDER = __CW + "../src/"
__DATA_FOLDER = __CW + "../data/"
__DATASET_FOLDER = __DATA_FOLDER + "datasets/"
__MODELS_FOLDER = __DATA_FOLDER + "models/"
__OUTPUT_FOLDER = __DATA_FOLDER + "output/"

sys.path.append(__SRC_FOLDER)

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

__DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
__SEED = 42

from models.transformer import VictoriaLoader
from xai.feature_importance import GlobalExplainer

## Loading utilities

In [ ]:
def load_model(output_file: str, model_type: str):
    """

    Args:
        output_file: The base output file name, without extension
        model_type: The type of model: one of "lr", "svm", "transformer"

    Returns:
        The trained model and its training parameters, if any.
    """
    if model_type == ["lr", "svm"]:
        with open(f"{output_file}.pickle", "rb") as log:
            model = pickle.load(log)
    elif model_type == "transformer":
        model = AutoModelForSequenceClassification()
        model.load_state_dict(torch.load(f"{output_file}.state_dict.pt"))
    else:
        raise ValueError(f"Unknown model {model_type}")

    return model

def load_victoria(task: Optional[str] = None, device: str = "cuda") -> Union[VictoriaLoader, pandas.DataFrame]:
    """
    Load the Victoria dataset.

    Args:
        task: The target task, if any
    Returns:
        A VictoriaLoader, if the `task` is specified, a raw DataFrame otherwise.
    """
    data = pandas.read_csv(__DATASET_FOLDER + "Gungor_2018_VictorianAuthorAttribution_data-train.csv",
                           encoding="latin-1")
    if task is not None:
        return VictoriaLoader(data, task, tokenizer=AutoTokenizer.from_pretrained("bart-base-uncased"), device=__DEVICE)
    else:
        return data

# Explaining linear models

In [ ]:
from models.preprocessing import preprocess_for_task
from models.preprocessing import n_grams

victoria_raw = load_victoria(device=__DEVICE)
character_n_grams = 3

## Feature importance

In [ ]:
task = "aa"
data = preprocess_for_task(victoria_raw, task,
                           sampling_size=1.,
                           negative_sampling_size=1.,
                           scale_labels=True,
                           seed=__SEED)
data, labels, vectorizer = n_grams(data, ngrams=character_n_grams, task=task)

# load model and explainer
model = load_model(f"{__MODELS_FOLDER}victoria_aa_svm.pickle", "svm")
explainer = GlobalExplainer(model, labels, data, task=task)

# explain
idx = 42
x_to_explain = data[idx]
_, explanation = explainer.explain(x_to_explain, data)
features_by_importance = numpy.flip(numpy.argsort(explanation))
# TODO: update w/ get_feature_names_out()
n_grams_by_importance = [vectorizer.vocabulary_[feature] for feature in features_by_importance]
n_gram_occurences_in_text = [[start for start in range(len(x_to_explain) - character_n_grams + 1) if data.startswith(n_gram, start)] for n_gram in n_grams_by_importance]
nr_n_gram_occurences_in_text = [len(occurences) for occurences in n_gram_occurences_in_text]

explanation_dataframe = pandas.DataFrame([(n_gram, explanation[i], i, nr_occurences) for i, (n_gram, nr_occurences) in enumerate(zip(n_grams_by_importance, nr_n_gram_occurences_in_text))],
                                         columns=("n_gram", "importance_score", "importance_rank", "nr_occurences_in_original_document"))
explanation_dataframe

In [6]:
task = "av"
data = preprocess_for_task(victoria_raw, task,
                           sampling_size=1.,
                           negative_sampling_size=1.,
                           scale_labels=True,
                           seed=__SEED)
data, labels, vectorizer = n_grams(data, ngrams=character_n_grams, task=task)

# load model and explainer
model = load_model(f"{__MODELS_FOLDER}victoria_av_svm", "svm")
explainer = GlobalExplainer(model, labels, data, task=task)

# explain
idx = 42
x_to_explain = data[idx]
_, explanation = explainer.explain(x_to_explain, data)
features_by_importance = numpy.flip(numpy.argsort(explanation))
n_grams_by_importance = [vectorizer.vocabulary_[feature] for feature in features_by_importance]
n_gram_occurences_in_text = [[start for start in range(len(x_to_explain) - character_n_grams + 1) if data.startswith(n_gram, start)] for n_gram in n_grams_by_importance]
nr_n_gram_occurences_in_text = [len(occurences) for occurences in n_gram_occurences_in_text]

explanation_dataframe = pandas.DataFrame([(n_gram, explanation[i], i, nr_occurences)
                                          for i, (n_gram, nr_occurences) in enumerate(zip(n_grams_by_importance, nr_n_gram_occurences_in_text))],
                                         columns=("n_gram", "importance_score", "importance_rank", "nr_occurences_in_original_document"))
explanation_dataframe

False

## Counterfactual example

In [ ]:
from xai.records import LocalCounterfactualExplainer

task = "aa"
data = preprocess_for_task(victoria_raw, task,
                           sampling_size=1.,
                           negative_sampling_size=1.,
                           scale_labels=True,
                           seed=__SEED)
data, labels, vectorizer = n_grams(data, ngrams=character_n_grams, task=task)

# load model and explainer
model = load_model(f"{__MODELS_FOLDER}victoria_aa_svm", "svm")
explainer = LocalCounterfactualExplainer(model, labels, data, task=task)

# explain
idx = 42
x_to_explain = data[idx]
counterfactual_x, counterfactual_x_prediction, counterfactual_distance = explainer.explain(x_to_explain, data)
feature_differences = counterfactual_x - x_to_explain
features_by_importance = numpy.flip(numpy.argsort(feature_differences))
n_grams_by_importance = [vectorizer.vocabulary_[feature] for feature in features_by_importance]

counterfactual_n_gram_occurences_in_text = [[start for start in range(len(x_to_explain) - character_n_grams + 1) if data.startswith(n_gram, start)] for n_gram in n_grams_by_importance]
nr_counterfactual_n_gram_occurences_in_text = [len(occurences) for occurences in counterfactual_n_gram_occurences_in_text]

explanation_dataframe = pandas.DataFrame([(n_gram, explanation[i], feature_differences[features_by_importance[i]], i, nr_occurences)
                                          for i, (n_gram, nr_occurences) in enumerate(zip(n_grams_by_importance, nr_counterfactual_n_gram_occurences_in_text))],
                                         columns=("n_gram", "importance_score", "counterfactual_distance_on_ngram", "importance_rank", "nr_occurences_in_original_document"))
explanation_dataframe

## Factual example

In [ ]:
from xai.records import LocalFactualExplainer

task = "aa"
data = preprocess_for_task(victoria_raw, task,
                           sampling_size=1.,
                           negative_sampling_size=1.,
                           scale_labels=True,
                           seed=__SEED)
data, labels, vectorizer = n_grams(data, ngrams=character_n_grams, task=task)

# load model and explainer
model = load_model(f"{__MODELS_FOLDER}victoria_aa_svm", "svm")
explainer = LocalFactualExplainer(model, labels, data, task=task)

# explain
idx = 42
x_to_explain = data[idx]
factual_x, factual_distance = explainer.explain(x_to_explain, data)
feature_differences = factual_x - x_to_explain
features_by_importance = numpy.argsort(feature_differences)
n_grams_by_importance = [vectorizer.vocabulary_[feature] for feature in features_by_importance]

factual_n_gram_occurences_in_text = [[start for start in range(len(x_to_explain) - character_n_grams + 1) if data.startswith(n_gram, start)] for n_gram in n_grams_by_importance]
nr_factual_n_gram_occurences_in_text = [len(occurences) for occurences in factual_n_gram_occurences_in_text]

explanation_dataframe = pandas.DataFrame([(n_gram, explanation[i], feature_differences[features_by_importance[i]], i, nr_occurences)
                                          for i, (n_gram, nr_occurences) in enumerate(zip(n_grams_by_importance, factual_n_gram_occurences_in_text))],
                                         columns=("n_gram", "importance_score", "factual_distance_on_ngram", "importance_rank", "nr_occurences_in_original_document"))
explanation_dataframe

# Explaining Transformers

In [2]:
import json

import lux

from xai.probing import TransformerProber

task = "aa"
model = load_model(f"{__MODELS_FOLDER}victoria_aa_transformer", "transformer")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
prober = TransformerProber(model, tokenizer)
prober.device = __DEVICE
victoria_dataloader = load_victoria("aa", device=__DEVICE)

NameError: name 'load_model' is not defined

## POS

In [ ]:
probe_type = "pos"
k, min_len, max_len = 10, 5, 10
n_jobs = -1

# probing
probing_results = prober.probe(victoria_dataloader,
                               probe_type=probe_type,
                               k=k, min_len=min_len, max_len=max_len,
                               n_jobs=n_jobs)

table_results = list()
for author, author_probe_results in probing_results.items():
    for i, (pos_chain, pos_chain_probing_results) in enumerate(author_probe_results):
        probe_model, probe_configuration, probe_validation = pos_chain_probing_results

        # store probing results
        with open(f"{__OUTPUT_FOLDER}probing_results/transformer_aa_pos_{i}.pickle", "wb") as log:
            pickle.dump(probe_model, log)
        with open(f"{__OUTPUT_FOLDER}probing_results/transformer_aa_pos_{i}.json", "w") as log:
            json.dump(probe_configuration, log)
        with open(f"{__OUTPUT_FOLDER}probing_results/transformer_aa_pos_{i}.validation.json", "w") as log:
            json.dump(probe_validation, log)

        # validation dataframe
        table_results.append([pos_chain, author,
                              probe_validation["1"]["precision"],
                              probe_validation["1"]["recall"],
                              probe_validation["1"]["f1-score"],
                              probe_validation["1"]["support"],
                              probe_validation["0"]["precision"],
                              probe_validation["0"]["recall"],
                              probe_validation["0"]["f1-score"],
                              probe_validation["0"]["support"]])
# dump validation table
pos_probe_columns=["pos_chain", "probed_author",
                   "precision_on_probe_success", "recall_on_probe_success", "f1score_on_probe_success", "support_on_probe_success",
                   "precision_on_probe_failure", "recall_on_probe_failure", "f1score_on_probe_failure", "support_on_probe_failure"]
results_dataframe = pandas.DataFrame(table_results, columns=pos_probe_columns)
results_dataframe.to_csv(f"{__OUTPUT_FOLDER}probing_results/transformer_aa_pos_probing.csv", index=False)

results_dataframe

## Lemmas/Stems/NER

In [ ]:
probe_type = "lemmas"
k, min_len, max_len = 10, 5, 10
n_jobs = -1

# probing
probing_results = prober.probe(victoria_dataloader,
                               probe_type="pos",
                               k=k, min_len=min_len, max_len=max_len,
                               n_jobs=n_jobs)

table_results = list()
for author, author_probe_results in probing_results.items():
    for i, (pos_chain, pos_chain_probing_results) in enumerate(author_probe_results):
        probe_model, probe_configuration, probe_validation = pos_chain_probing_results

        # store probing results
        with open(f"{__OUTPUT_FOLDER}probing_results/transformer_aa_pos_{i}.pickle", "wb") as log:
            pickle.dump(probe_model, log)
        with open(f"{__OUTPUT_FOLDER}probing_results/transformer_aa_pos_{i}.json", "w") as log:
            json.dump(probe_configuration, log)
        with open(f"{__OUTPUT_FOLDER}probing_results/transformer_aa_pos_{i}.validation.json", "w") as log:
            json.dump(probe_validation, log)

        # validation dataframe
        table_results.append([pos_chain, author,
                              probe_validation["1"]["precision"],
                              probe_validation["1"]["recall"],
                              probe_validation["1"]["f1-score"],
                              probe_validation["1"]["support"]])
# dump validation table
pos_probe_columns=["pos_chain", "probed_author",
                   "precision_on_probe_success", "recall_on_probe_success", "f1score_on_probe_success", "support_on_probe_success",
                   "precision_on_probe_failure", "recall_on_probe_failure", "f1score_on_probe_failure", "support_on_probe_failure"]
results_dataframe = pandas.DataFrame(table_results, columns=pos_probe_columns)
results_dataframe.to_csv(f"{__OUTPUT_FOLDER}probing_results/transformer_aa_pos_probing.csv", index=False)

results_dataframe